1. Здесь идет установка **необходимых библиотек**

In [ ]:
!pip3 install pydub
!pip3 install vosk
!pip3 install torch
!pip3 install transformers
!git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg
%pip install moviepy

1.2 Vosk model installation

In [16]:
!curl -o ./model.zip https://alphacephei.com/vosk/models/vosk-model-ru-0.42.zip
!unzip model.zip
!mv vosk-model-ru-0.42/ model
!rm -rf model.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1847M  100 1847M    0     0  3046k      0  0:10:21  0:10:21 --:--:-- 3121k 0  3018k      0  0:10:26  0:00:25  0:10:01 3596k8:05 5770k0  0:06:02  0:02:29  0:03:33 4688k   0  4887k      0  0:06:27  0:03:19  0:03:08 4098k    0  4214k      0  0:07:28  0:04:49  0:02:39 1851k  0     0  3658k      0  0:08:37  0:06:45  0:01:52 2032k   0     0  3519k      0  0:08:57  0:07:20  0:01:37 1570k05  0:07:33  0:01:32 1977k     0  3114k      0  0:10:07  0:09:22  0:00:45 1170k
Archive:  model.zip
   creating: vosk-model-ru-0.42/
   creating: vosk-model-ru-0.42/graph/
  inflating: vosk-model-ru-0.42/graph/words.txt  
   creating: vosk-model-ru-0.42/graph/phones/
 extracting: vosk-model-ru-0.42/graph/phones/silence.csl  
  inflating: vosk-model-ru-0.42/graph/phones/align_lexicon.txt  
  inflating: vosk-model-ru-0.42/graph/phones/word_boundary.txt 

This model can be put to VOSK, but its use takes too much resources of the computer itself. Deals with punctuation marks

In [ ]:
!curl -o recasepunc.zip https://alphacephei.com/vosk/models/vosk-recasepunc-ru-0.22.zip
!unzip recasepunc.zip
!mv vosk-recasepunc-ru-0.22/ recasepunc
!rm -rf recasepunc.zip

In [25]:
!git clone https://github.com/benob/recasepunc
!pip3 install -r ./recasepunc/requirements.txt -f https://download.pytorch.org/whl/torch_stable.html

fatal: destination path 'recasepunc' already exists and is not an empty directory.
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Cloning https://github.com/benob/mosestokenizer.git to /tmp/pip-req-build-k83wezdf
  Running command git clone --filter=blob:none --quiet https://github.com/benob/mosestokenizer.git /tmp/pip-req-build-k83wezdf
  Resolved https://github.com/benob/mosestokenizer.git to commit 169bd3a504fe20a3e51b9a7af3f0ca359c2d36c9
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.19.5.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached regex-2021.8.28-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (39 kB)
ERROR: Could not find a version that satisfies the requirement torch==1.9.0+cu111 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu1

In [26]:
#import libraries
from vosk import Model, KaldiRecognizer, SetLogLevel
from pydub import AudioSegment
import subprocess
import json
import os
from moviepy.editor import VideoFileClip
from moviepy.video.io.html_tools import ffmpeg_parse_infos


In [27]:
from moviepy.editor import VideoFileClip
def convert_mp4_to_wav(input_file, output_file):
    video = VideoFileClip(input_file)
    audio = video.audio
    audio.write_audiofile(output_file)

input_file = 'video.mp4'
output_file = 'file.wav'
convert_mp4_to_wav(input_file, output_file)


MoviePy - Writing audio in file.wav


MoviePy - Done.


In [28]:
#SetLogLevel(0)
import os
# checking model
if not os.path.exists("model"):
    print ("Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in the current folder.")
    exit (1)

# set Frame Rate
FRAME_RATE = 16000
CHANNELS=1

model = Model("model")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

# Используя библиотеку pydub делаем предобработку аудио
mp3 = AudioSegment.from_mp3('file.wav')
mp3 = mp3.set_channels(CHANNELS)
mp3 = mp3.set_frame_rate(FRAME_RATE)


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from model/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from model/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:297) Loading words from model/graph/words.txt
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo model/graph/phones/word_boundary.int
LOG (VoskAPI:ReadDataFiles():model.cc:315) Loading sub

In [ ]:
# Преобразуем вывод в json
rec.AcceptWaveform(mp3.raw_data)
result = rec.Result()
text = json.loads(result)["text"]

# Добавляем пунктуацию (Занимает очень много ресурсов, можно и без нее)
#cased = subprocess.check_output('python3 recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)
# Записываем результат в файл "data.txt"
with open('ff.txt', 'w') as f:
    json.dump(text, f, ensure_ascii=False, indent=4)

In [36]:
#@title Install dependencies

import os
import yaml
import torch
from torch import package

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)

with open('latest_silero_models.yml', 'r') as yaml_file:
    models = yaml.load(yaml_file, Loader=yaml.SafeLoader)
model_conf = models.get('te_models').get('latest')

In [ ]:
# see avaiable languages
available_languages = list(model_conf.get('languages'))
print(f'Available languages {available_languages}')

# and punctuation marks
available_punct = list(model_conf.get('punct'))
print(f'Available punctuation marks {available_punct}')

In [ ]:
model_url = model_conf.get('package')

model_dir = "downloaded_model"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, os.path.basename(model_url))

if not os.path.isfile(model_path):
    torch.hub.download_url_to_file(model_url,
                                   model_path,
                                   progress=True)

imp = package.PackageImporter(model_path)
model = imp.load_pickle("te_model", "model")
example_texts = model.examples

def apply_te(text, lan='ru'):
    return model.enhance_text(text, lan)

In [ ]:
input_text = model.examples[0]
output_text = apply_te(input_text, lan='ru')
print(f"Input: \n{input_text}\nOutput:\n{output_text}") 

In [ ]:
with open('ff.txt', 'r', encoding='utf-8') as file:
    input_text = file.read()
    
#input_text = input('Enter input text\n')
print(apply_te(input_text, lan='ru'))

